In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv("C:/Users/Nikanshi/Documents/Projects/ab_data.csv")

In [4]:
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
user_id         294478 non-null int64
timestamp       294478 non-null object
group           294478 non-null object
landing_page    294478 non-null object
converted       294478 non-null int64
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [7]:
df['user_id'].nunique()

290584

In [9]:
df.converted.count()

294478

In [10]:
df[((df['group'] == 'treatment') == (df['landing_page'] == 'new_page')) == False].count()

user_id         3893
timestamp       3893
group           3893
landing_page    3893
converted       3893
dtype: int64

In [12]:
# Where treatment does not match with new_page or control does not match with old_page
i = df[((df['group']=='treatment') ==(df['landing_page']=='new_page')) == False].index

In [13]:
# Drop those rows
df = df.drop(i)

In [15]:
df[((df['group'] == 'treatment') == (df['landing_page'] == 'new_page')) == False].shape[0]

0

In [16]:
#find duplicate user ids
df[df.duplicated(['user_id'], keep=False)]

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [17]:
#drop one of the duplicate records
df.drop_duplicates(subset ='user_id',keep ='first',inplace = True)

In [21]:
#Conversion Probabilities
overall = (df.query('converted == 1').converted.count())/df.shape[0]

treatment_df = df.query('group =="treatment"')
new = treatment_df['converted'].mean()

control_df = df.query('group =="control"')
old = control_df['converted'].mean()

print('Overall Conversion Probability: ' + str(overall))
print('Conversion Probability for New Page: ' + str(new))
print('Conversion Probability for Old Page: '+ str(old))

Overall Conversion Probability: 0.11959708724499628
Conversion Probability for New Page: 0.11880806551510564
Conversion Probability for Old Page: 0.1203863045004612


In [23]:
# Prob of visiting the new page
df.query('landing_page == "new_page"').landing_page.count()/df.shape[0]


0.5000619442226688

# Testing Hypothesis - Z Score 

In [24]:
import statsmodels.api as sm

convert_old = df.query("landing_page == 'old_page' and converted == 1").shape[0]
convert_new = df.query("landing_page == 'new_page' and converted == 1").shape[0]
n_old = df.query("landing_page == 'old_page'").shape[0]
n_new = df.query("landing_page == 'new_page'").shape[0]

In [25]:
z_score, p_value = sm.stats.proportions_ztest([convert_old, convert_new], [n_old, n_new], alternative='smaller')
z_score, p_value

(1.3109241984234394, 0.9050583127590245)

Since our p-value is larger than 0.05  we fail to reject our null hypothesis

# Testing Hypothesis - Regression

In [26]:
df['ab_page'] = pd.get_dummies(df['group']) ['treatment']
df.head()

,user_id,timestamp,group,landing_page,converted,ab_page
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,1
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,1
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,0


In [27]:
df['intercept'] = 1
logit_mod = sm.Logit(df['converted'],df[['intercept', 'ab_page']])
results = logit_mod.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.366118
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               290584
Model:                          Logit   Df Residuals:                   290582
Method:                           MLE   Df Model:                            1
Date:                Wed, 20 May 2020   Pseudo R-squ.:               8.077e-06
Time:                        21:45:21   Log-Likelihood:            -1.0639e+05
converged:                       True   LL-Null:                   -1.0639e+05
                                        LLR p-value:                    0.1899
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -1.9888      0.008   -246.669      0.000      -2.005      -1.973
ab_page       -0.0150      0.011     -1.311      0.190      -0.037       0.007
==============================================================================
"""

p-value is 0.190 hence, we again fail to reject the null hypothesis

There is no significant differnce in conversion rates for new landing page and old landing page